In [2]:
import keras
from keras.datasets import california_housing

In [3]:
(train_features,train_labels),(test_features,test_labels) = california_housing.load_data(version='small')

743530/743530 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
